In [7]:
# Note: This was used for experimentation, the final version used to generate output is found
# in clustering.py in this directory.

# Dependency management and python versions are found in pyproject.toml. this was run with phttps://python-poetry.org/
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity
import os
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# Load the data, from ../../data/2_raw_text
data = []
names = []
for root, dirs, files in os.walk("../../data/2_raw_text"):
    print(f"-Dir: {root}")
    for dir in dirs:
        print(f"--{dir}")
    for filename in files:
        print(f"---{filename}")
        with open(f"{root}/{filename}") as f:
            data.append(f.read())
            names.append(filename)

-Dir: ../../data/2_raw_text
---1000_text.txt
---1001_text.txt
---1002_text.txt
---1003_text.txt
---1004_text.txt
---1005_text.txt
---1006_text.txt
---1007_text.txt
---1008_text.txt
---1009_text.txt
---100_text.txt
---1010_text.txt
---1011_text.txt
---1012_text.txt
---1013_text.txt
---1014_text.txt
---1015_text.txt
---1016_text.txt
---1017_text.txt
---1018_text.txt
---1019_text.txt
---101_text.txt
---1020_text.txt
---1021_text.txt
---1022_text.txt
---1023_text.txt
---1024_text.txt
---1025_text.txt
---1026_text.txt
---1027_text.txt
---1028_text.txt
---1029_text.txt
---102_text.txt
---1030_text.txt
---1031_text.txt
---1032_text.txt
---1033_text.txt
---1034_text.txt
---1035_text.txt
---1036_text.txt
---1037_text.txt
---1038_text.txt
---1039_text.txt
---103_text.txt
---1040_text.txt
---1042_text.txt
---1043_text.txt
---1044_text.txt
---1045_text.txt
---1046_text.txt
---1047_text.txt
---1048_text.txt
---1049_text.txt
---104_text.txt
---1050_text.txt
---1051_text.txt
---1052_text.txt
---1053_

---1454_text.txt
---1455_text.txt
---1456_text.txt
---1457_text.txt
---1458_text.txt
---1459_text.txt
---145_text.txt
---1460_text.txt
---1461_text.txt
---1462_text.txt
---1463_text.txt
---1464_text.txt
---1465_text.txt
---1466_text.txt
---1467_text.txt
---1468_text.txt
---1469_text.txt
---146_text.txt
---1470_text.txt
---1471_text.txt
---1472_text.txt
---1473_text.txt
---1474_text.txt
---1475_text.txt
---1476_text.txt
---1477_text.txt
---1478_text.txt
---1479_text.txt
---147_text.txt
---1480_text.txt
---1481_text.txt
---1482_text.txt
---1483_text.txt
---1484_text.txt
---1485_text.txt
---1486_text.txt
---1487_text.txt
---1488_text.txt
---1489_text.txt
---148_text.txt
---1490_text.txt
---1491_text.txt
---1492_text.txt
---1493_text.txt
---1494_text.txt
---1495_text.txt
---1496_text.txt
---1497_text.txt
---1498_text.txt
---1499_text.txt
---149_text.txt
---1500_text.txt
---1501_text.txt
---1502_text.txt
---1503_text.txt
---1504_text.txt
---1506_text.txt
---1508_text.txt
---1509_text.txt
--

---57_text.txt
---580_text.txt
---581_text.txt
---582_text.txt
---583_text.txt
---584_text.txt
---585_text.txt
---586_text.txt
---587_text.txt
---588_text.txt
---589_text.txt
---58_text.txt
---590_text.txt
---591_text.txt
---592_text.txt
---593_text.txt
---594_text.txt
---595_text.txt
---596_text.txt
---597_text.txt
---598_text.txt
---599_text.txt
---59_text.txt
---5_text.txt
---600_text.txt
---601_text.txt
---602_text.txt
---603_text.txt
---604_text.txt
---605_text.txt
---606_text.txt
---607_text.txt
---608_text.txt
---609_text.txt
---60_text.txt
---610_text.txt
---611_text.txt
---612_text.txt
---613_text.txt
---614_text.txt
---615_text.txt
---616_text.txt
---617_text.txt
---618_text.txt
---619_text.txt
---61_text.txt
---620_text.txt
---621_text.txt
---622_text.txt
---623_text.txt
---624_text.txt
---625_text.txt
---626_text.txt
---627_text.txt
---628_text.txt
---629_text.txt
---62_text.txt
---630_text.txt
---631_text.txt
---632_text.txt
---633_text.txt
---634_text.txt
---635_text.txt


In [9]:
print(len(data))

1608


In [10]:
# Example data point, showing the raw text that we'll embed with BERT
# and attempt to cluster
print(data[0])








117th CONGRESS
  1st Session
                                H. R. 1000

    To amend title XVIII of the Social Security Act to provide for 
    infection control support to skilled nursing facilities through 
           contracts with quality improvement organizations.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           February 11, 2021

 Mr. Kind (for himself and Mr. Kildee) introduced the following bill; 
which was referred to the Committee on Ways and Means, and in addition 
      to the Committee on Energy and Commerce, for a period to be 
subsequently determined by the Speaker, in each case for consideration 
  of such provisions as fall within the jurisdiction of the committee 
                               concerned

_______________________________________________________________________

                                 A BILL


 
    To amend title XVIII of the Soc

In [11]:
# Create Embeddings from the raw data, using distilbert
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(data, show_progress_bar=True)

Batches:   0%|          | 0/51 [00:00<?, ?it/s]

In [12]:
# Use UMAP to lower the dimensions of the data for HDBScan. 
# DBScan based algorithms don't work well with high dimensional data
reduced_embeddings = umap.UMAP(n_neighbors=5, 
                               n_components=5, 
                               metric='cosine').fit_transform(embeddings)

In [13]:
# Run HDBScan on the reduced embeddings
cluster = hdbscan.HDBSCAN(min_cluster_size=3,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(reduced_embeddings)
cluster

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=3, min_samples=None, p=None,
        prediction_data=False)

In [18]:
# Load the bills into a data frame, with the topic numbers and some metadata
df = pd.DataFrame(data, columns=["Bill"])
df['Topic'] = cluster.labels_
df['Bill_Id'] = range(len(df))
bills_per_topic = df.groupby(['Topic'], as_index = False).agg({'Bill': ' '.join})
df["Bill_Number"] = [i.split("_")[0] for i in names]
df["File_Name"] = names
df.head(10)

,Bill,Topic,Bill_Id,Bill_Number,File_Name
0,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,0,1000,1000_text.txt
1,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,120,1,1001,1001_text.txt
2,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,2,1002,1002_text.txt
3,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,71,3,1003,1003_text.txt
4,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,28,4,1004,1004_text.txt
5,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,5,1005,1005_text.txt
6,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,129,6,1006,1006_text.txt
7,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,7,1007,1007_text.txt
8,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,129,8,1008,1008_text.txt
9,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,83,9,1009,1009_text.txt


In [113]:
# Sanitize the bills, removing non-alphabetical characters and stopwords
def bill_sanitizer(raw_bill):
    letters_only = re.sub('[^a-zA-Z]', ' ', raw_bill)
    words = letters_only.lower().split()
    stopwords_set = set(stopwords.words("english"))
    nonstopwords = [x for x in words if not x in stopwords_set]
    
    # Combine words into a paragraph again
    output_strings = ' '.join(nonstopwords)
    return(output_strings)
cleaned_bills_per_topic = [bill_sanitizer(i) for i in bills_per_topic.Bill.values]
cleaned_bills_per_topic[0]

'th congress st session h r amend title xviii social security act provide infection control support skilled nursing facilities contracts quality improvement organizations house representatives february mr kind mr kildee introduced following bill referred committee ways means addition committee energy commerce period subsequently determined speaker case consideration provisions fall within jurisdiction committee concerned bill amend title xviii social security act provide infection control support skilled nursing facilities contracts quality improvement organizations enacted senate house representatives united states america congress assembled section short title act may cited infection control training support act sec providing infection control support skilled nursing facilities contracts quality improvement organizations section g social security act u c g amended striking secretary inserting secretary adding end following new paragraph addition amounts otherwise available appropriat

In [98]:
def cluster_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = cluster_tf_idf(cleaned_bills_per_topic, m=len(data))
count

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [99]:
def extract_top_words(tf_idf, count, bills_per_topic, n=5):
    words = count.get_feature_names()
    labels = list(bills_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_cluster_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Bill
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Bill": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_words(tf_idf, count, bills_per_topic, n=10)
topic_sizes = extract_cluster_sizes(df)
topic_sizes.head(10)

,Topic,Size
0,-1,421
30,29,53
17,16,40
110,109,32
62,61,30
122,121,23
64,63,20
99,98,19
101,100,17
6,5,17


In [100]:
df.sort_values(by=['Topic'])

,Bill,Topic,Bill_Id,Bill_Number,File_Name
0,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,0,1000,1000_text.txt
900,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,900,360,360_text.txt
358,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,358,1327,1327_text.txt
909,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,909,369,369_text.txt
355,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,-1,355,1324,1324_text.txt
...,...,...,...,...,...
945,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,141,945,401,401_text.txt
464,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,141,464,1423,1423_text.txt
1178,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,141,1178,612,612_text.txt
768,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,141,768,248,248_text.txt


In [106]:
# List top clusters, and their sizes. Note: -1 in HDBScan is unclassified 
# documents, in this case bills that don't fall into a cluster
topic_sizes = extract_cluster_sizes(df)
topic_sizes.head(10)

,Topic,Size
0,-1,421
30,29,53
17,16,40
110,109,32
62,61,30
122,121,23
64,63,20
99,98,19
101,100,17
6,5,17


In [107]:
topics = [29,16,109,61,121,63,98,100,5]
for i in topics:
    print(i)
    print(top_n_words[i])

29
[('prohibit', 0.007284721086609355), ('senator', 0.005942518117267547), ('automobile', 0.005808141380396182), ('accommodations', 0.005777798249083911), ('coach', 0.005090989319549936), ('world', 0.00498582053066235), ('resolution', 0.004876723902880325), ('hostilities', 0.004836439853572439), ('airline', 0.004445413478509656), ('earmark', 0.003828126226867605)]
16
[('sales', 0.010497329271668198), ('said', 0.009681423213084671), ('administering', 0.007625866407890886), ('street', 0.00617955845689267), ('intersection', 0.00606934963245672), ('taxes', 0.00603942921814532), ('way', 0.005793936764786044), ('nw', 0.0056797390385508895), ('intermediation', 0.005401866478793824), ('imposed', 0.0048940908959190675)]
109
[('nmosd', 0.0073772998615601), ('substance', 0.00733639845421257), ('mental', 0.007264588164179719), ('behavioral', 0.0070388636668975236), ('tracing', 0.006398045929354287), ('disorder', 0.005687534763426636), ('vaccination', 0.005389458283863146), ('valvular', 0.005082133

In [108]:
# Filter bills from a specific topic, in this case 23 which seems to have a strong theme
bytopic = df['Topic']== 63
df[bytopic]

,Bill,Topic,Bill_Id,Bill_Number,File_Name
140,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,140,1129,1129_text.txt
142,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,142,1130,1130_text.txt
143,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,143,1131,1131_text.txt
144,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,144,1132,1132_text.txt
189,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,189,1173,1173_text.txt
198,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,198,1181,1181_text.txt
319,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,319,1291,1291_text.txt
714,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,714,2042,2042_text.txt
908,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,908,368,368_text.txt
935,\n\n\n\n\n\n\n117th CONGRESS\n 1st Session\n ...,63,935,393,393_text.txt


In [109]:
# List some bills from this topic
bill_ids = [140,142,189,1060]
print("Topic:", df["Topic"][140])
print("Topic Key Words:", top_n_words[df["Topic"][140]])
for bill_id in bill_ids:
    print("------------------------------------")
    print("Bill Id:", bill_id)
    print("------------------------------------")
    print(df['Bill'][bill_id][5:500])
    print("\n\n\n\n")


Topic: 63
Topic Key Words: [('taiwan', 0.0149630548410296), ('china', 0.013357374724206643), ('republic', 0.007346960419476861), ('protected', 0.007215421469050169), ('reprisal', 0.006442551738808892), ('flood', 0.006190893594219868), ('intellectual', 0.005864299320172789), ('chinese', 0.005487504891935479), ('borrower', 0.005322687901751229), ('press', 0.005006219142901557)]
------------------------------------
Bill Id: 140
------------------------------------


117th CONGRESS
  1st Session
                                H. R. 1129

  To require annual reporting on the availability of Federal funds to 
persons and entities of China and activities conducted in collaboration 
                  with China, and for other purposes.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           February 18, 2021

    Mr. Green of Tennessee introduced the follo





---------------------------------

In [110]:
# Drop the bill content and ID, since we only 
# care about topic and bill number / file name
df2 = df.drop("Bill",axis=1)
df2 = df2.drop("Bill_Id",axis=1)
print(df2)

      Topic Bill_Number      File_Name
0        -1        1000  1000_text.txt
1       120        1001  1001_text.txt
2        -1        1002  1002_text.txt
3        71        1003  1003_text.txt
4        28        1004  1004_text.txt
...     ...         ...            ...
1603    135         996   996_text.txt
1604     -1         997   997_text.txt
1605     39         998   998_text.txt
1606     90         999   999_text.txt
1607     36          99    99_text.txt

[1608 rows x 3 columns]


In [111]:
# Write out a map of what bills map to what topics
json_data = df2.to_json(orient="split")
import json
parsed = json.loads(json_data)
with open ("../../data/4_clustering/clustering_bills.json", "w") as f:
    json.dump(parsed, f)

In [112]:
# Write out a map of key words that map to each topic
with open ("../../data/4_clustering/topic_map.json", "w") as f:
    json.dump(top_n_words, f)